In [1]:
import os
import sys
import time
import logging
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch

c:\Users\Soumyabrata\anaconda3\envs\Tf2_Py37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from tf_pose import common
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh

In [3]:
model='mobilenet_thin'
resize='432x368'
w, h = model_wh(resize)

In [4]:
e = TfPoseEstimator(get_graph_path(model), target_size=(w, h))

[2023-07-05 15:11:39,392] [TfPoseEstimator] [INFO] loading graph from c:\Users\Soumyabrata\Pose_Estimation\tf-pose-estimation\models\graph/mobilenet_thin/graph_opt.pb(default size=432x368)
2023-07-05 15:11:39,392 INFO loading graph from c:\Users\Soumyabrata\Pose_Estimation\tf-pose-estimation\models\graph/mobilenet_thin/graph_opt.pb(default size=432x368)


TfPoseEstimator/MobilenetV1/Conv2d_0/weights
TfPoseEstimator/image
TfPoseEstimator/MobilenetV1/Conv2d_0/Conv2D
TfPoseEstimator/MobilenetV1/Conv2d_0/Conv2D_bn_offset
TfPoseEstimator/MobilenetV1/Conv2d_0/BatchNorm/FusedBatchNorm
TfPoseEstimator/MobilenetV1/Conv2d_0/Relu
TfPoseEstimator/MobilenetV1/Conv2d_1_depthwise/depthwise_weights
TfPoseEstimator/MobilenetV1/Conv2d_1_pointwise/weights
TfPoseEstimator/MobilenetV1/Conv2d_1_depthwise/depthwise
TfPoseEstimator/MobilenetV1/Conv2d_1_pointwise/Conv2D
TfPoseEstimator/MobilenetV1/Conv2d_1_pointwise/Conv2D_bn_offset
TfPoseEstimator/MobilenetV1/Conv2d_1_pointwise/BatchNorm/FusedBatchNorm
TfPoseEstimator/MobilenetV1/Conv2d_1_pointwise/Relu
TfPoseEstimator/MobilenetV1/Conv2d_2_depthwise/depthwise_weights
TfPoseEstimator/MobilenetV1/Conv2d_2_pointwise/weights
TfPoseEstimator/MobilenetV1/Conv2d_2_depthwise/depthwise
TfPoseEstimator/MobilenetV1/Conv2d_2_pointwise/Conv2D
TfPoseEstimator/MobilenetV1/Conv2d_2_pointwise/Conv2D_bn_offset
TfPoseEstimator/M

In [ ]:
root_path='/home/mtech/project_env/mmaction2/tools/data/ucf101/UCF101_frames'
out_dir='/home/mtech/project_env/mmaction2/tools/data/ucf101/UCF101_frames_pose'
cls_list=os.listdir(root_path)
for cls in cls_list:
    vid_list=os.listdir(os.path.join(root_path,cls))
    for vid in vid_list:
        frame_list=os.listdir(os.path.join(root_path,cls,vid))
        for i in range(len(frame_list)):
            if (i%10)==0 :
                image_path=os.path.join(root_path,cls,vid,frame_list[i])
                image_out=os.path.join(out_dir,cls,vid,frame_list[i].split('.')[0])+ '.npy'
                image = common.read_imgfile(image_path, None, None)
                humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=4.0)
                max_prob = np.amax(e.heatMat[:, :, :-1], axis=2)
                if not os.path.exists(os.path.join(out_dir,cls,vid)):
                    os.makedirs(os.path.join(out_dir,cls,vid))
                np.save(image_out,max_prob)